In [ ]:
!pip install hopsworks[python] prophet requests matplotlib pandas numpy scikit-learn


In [ ]:

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import hopsworks

FEATURE_GROUP_NAME = "aqi_weather_features"
FEATURE_GROUP_VER  = 2
LATITUDE  = 33.5973
LONGITUDE = 73.0479
HORIZON_H = 72
TZ = "Asia/Karachi"
ARTIFACT_DIR = "prophet_aqi_artifacts"
PLOTS_DIR    = os.path.join(ARTIFACT_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# ------------------------
# 1) Load data
# ------------------------
print("[1/6] Loading Feature Group from Hopsworks...")
project = hopsworks.login()
fs = project.get_feature_store()
fg = fs.get_feature_group(name=FEATURE_GROUP_NAME, version=FEATURE_GROUP_VER)
df_raw = fg.read()

df_raw = df_raw.sort_values("time", ascending=True).reset_index(drop=True)
df_raw["time"] = pd.to_datetime(df_raw["time"])
df_raw["time"] = df_raw["time"].dt.tz_localize(None)  # Prophet requires naive datetime

# Prepare Prophet format
df_prophet = df_raw[["time", "us_aqi"]].rename(columns={"time": "ds", "us_aqi": "y"})

# ------------------------
# 2) Train Prophet
# ------------------------
print("[2/6] Training Prophet model...")
model = Prophet(daily_seasonality=True, yearly_seasonality=True)
model.fit(df_prophet)

# ------------------------
# 3) Forecast
# ------------------------
print("[3/6] Forecasting next 72h...")
future = model.make_future_dataframe(periods=HORIZON_H, freq="H")
forecast = model.predict(future)

# Merge with actual AQI (if available)
forecast_df = forecast[["ds", "yhat"]].tail(HORIZON_H).copy()
forecast_df["ds"] = forecast_df["ds"].dt.tz_localize("UTC").dt.tz_convert(TZ)
forecast_df.rename(columns={"ds": "datetime", "yhat": "predicted_us_aqi"}, inplace=True)

forecast_path = os.path.join(ARTIFACT_DIR, "prophet_72h_forecast.csv")
forecast_df.to_csv(forecast_path, index=False)

# ------------------------
# 4) Plot
# ------------------------
print("[4/6] Saving forecast plot...")
fig1 = model.plot(forecast)
fig1.savefig(os.path.join(PLOTS_DIR, "prophet_forecast.png"), dpi=140)
plt.close(fig1)

# ------------------------
# 5) Evaluate (last 20% of history)
# ------------------------
print("[5/6] Evaluating Prophet on last 20% history...")
split_idx = int(len(df_prophet) * 0.8)
test_df = df_prophet.iloc[split_idx:]
pred_test = model.predict(test_df.rename(columns={"time": "ds"}))
y_true = test_df["y"].values
y_pred = pred_test["yhat"].values
mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)
print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.4f}")

print("\n✅ Prophet pipeline done.")


[1/6] Loading Feature Group from Hopsworks...
Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated

Paste it here: ··········




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1239199
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.07s) 
[2/6] Training Prophet model...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpa57sxwey/oxjphm_2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpa57sxwey/qg43t6qa.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99102', 'data', 'file=/tmp/tmpa57sxwey/oxjphm_2.json', 'init=/tmp/tmpa57sxwey/qg43t6qa.json', 'output', 'file=/tmp/tmpa57sxwey/prophet_model351scvs4/prophet_model-20250815224909.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
22:49:09 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
22:49:57 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


[3/6] Forecasting next 72h...
[4/6] Saving forecast plot...
[5/6] Evaluating Prophet on last 20% history...
MAE: 25.22, RMSE: 955.85, R²: 0.2132

✅ Prophet pipeline done.


In [ ]:
# prophet_forecast.py
import requests
import pandas as pd
import joblib
from prophet import Prophet
from datetime import datetime, timedelta, timezone

# # === Load trained Prophet model ===
# MODEL_PATH = "prophet_model.pkl"
# model = joblib.load(MODEL_PATH)

# === Config ===
lat, lon = 33.6844, 73.0479  # Example: Islamabad
forecast_hours = 72

# === Step 1: Fetch AQI + Weather Forecast from Open-Meteo ===
aqi_url = (
    "https://air-quality-api.open-meteo.com/v1/air-quality"
    f"?latitude={lat}&longitude={lon}&hourly=pm2_5"
)


aqi_data = requests.get(aqi_url).json()

# AQI DataFrame
df_aqi = pd.DataFrame({
    "datetime": aqi_data["hourly"]["time"],
    "aqi": aqi_data["hourly"]["pm2_5"]
})



# Merge on datetime
# df_forecast = pd.merge(df_aqi, df_weather, on="datetime")

# Convert datetime to pandas datetime
df_aqi["datetime"] = pd.to_datetime(df_aqi["datetime"])

# === Step 2: Prepare for Prophet ===
# Prophet expects columns: ds (datetime), y (target)
# But for forecast, we only need ds; model handles the rest.
future_df = df_aqi.rename(columns={"datetime": "ds"})

# === Step 3: Predict ===
forecast = model.predict(future_df[["ds"]])

# === Step 4: Merge predictions with input features ===
result = pd.concat([future_df, forecast[["yhat", "yhat_lower", "yhat_upper"]]], axis=1)

print("\n=== Prophet Forecast ===")
print(result.head(forecast_hours))



=== Prophet Forecast ===
                    ds   aqi  temperature  humidity  wind_speed         yhat  \
0  2025-08-15 00:00:00   6.8         24.9        90         9.9 -3344.088785   
1  2025-08-15 01:00:00  10.8         24.7        90         9.8 -3398.458789   
2  2025-08-15 02:00:00  15.8         24.7        91         9.5 -3436.712749   
3  2025-08-15 03:00:00  16.5         24.7        92        10.1 -3446.907888   
4  2025-08-15 04:00:00  14.2         24.9        92        10.5 -3415.854362   
..                 ...   ...          ...       ...         ...          ...   
67 2025-08-17 19:00:00  50.6         26.0        92         4.2 -2357.327089   
68 2025-08-17 20:00:00  42.1         25.8        93         4.2 -2246.291826   
69 2025-08-17 21:00:00  39.1         25.7        94         4.6 -2127.770840   
70 2025-08-17 22:00:00  36.6         25.6        94         3.6 -2007.331069   
71 2025-08-17 23:00:00  24.3         25.4        94         3.5 -1887.078195   

     yhat_low